In [1]:
import numpy as np
from dask import delayed
from dask.distributed import Client, progress
import dask.array as da

import warnings
warnings.filterwarnings("ignore")

/Users/cri/anaconda3/envs/PythonData/lib/python3.6/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/Users/cri/anaconda3/envs/PythonData/lib/python3.6/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [2]:
client = Client(n_workers=4, threads_per_worker=2, memory_limit='2GB')
client

Client Scheduler: tcp://127.0.0.1:49317 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 8.00 GB


Time without persisting the array

In [3]:
%%timeit
x = da.random.random((10000, 10000), chunks=(1000, 1000))
y = x + x.T
z = y[::2, 5000:].mean(axis=1)
z.compute()

525 ms ± 32.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Time with persisting the array

In [4]:
x = da.random.random((10000, 10000), chunks=(1000,1000))

In [5]:
x.persist()

dask.array<random_sample, shape=(10000, 10000), dtype=float64, chunksize=(1000, 1000)>

In [6]:
%%time
y = x + x.T
z = y[::2, 5000:].mean(axis=1)
z.compute()

CPU times: user 170 ms, sys: 22.2 ms, total: 193 ms
Wall time: 387 ms


Time with chunks set to 250, 250

In [7]:
x = da.random.random((10000, 10000), chunks=(250,250))

In [8]:
x.persist()

dask.array<random_sample, shape=(10000, 10000), dtype=float64, chunksize=(250, 250)>

In [9]:
%%time
y = x + x.T
z = y[::2, 5000:].mean(axis=1)
z.compute()

CPU times: user 1.09 s, sys: 75.1 ms, total: 1.17 s
Wall time: 1.25 s


Time with chunks 500, 500

In [10]:
x = da.random.random((10000, 10000), chunks=(500,500))

In [11]:
x.persist()

dask.array<random_sample, shape=(10000, 10000), dtype=float64, chunksize=(500, 500)>

In [12]:
%%time
y = x + x.T
z = y[::2, 5000:].mean(axis=1)
z.compute()

CPU times: user 363 ms, sys: 31.8 ms, total: 395 ms
Wall time: 661 ms


### Discussion
Dask is not as fast at Numpy. However it would allow for arrays larger than a single computer's memory. Here we see the drawbacks of segmenting the information across several arrays as Dask does. As chunksize decreases processing time increases because accessing each array adds additional steps rather than if only one array was being read.